# Tensorboard

## Usage

Tensorboard is a visualization toolkit for machine learning experimentation. Originally from tensorflow, but PyTorch has a utility package that lets us use tensorboard with PyTorch.

## Code Tutorial

In [2]:
import os
import shutil
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Download data

In [3]:
transforms = torchvision.transforms.Compose([
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.Normalize(
                                                 (0.1307,), (0.3081,)
                                             )
])

train_set = torchvision.datasets.MNIST('./mnist-data', train=True, download=True,
                                     transform=transforms)
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=64, shuffle=True)

test_set = torchvision.datasets.MNIST('./mnist-data', train=False, download=True,
                                    transform=transforms)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=1000, shuffle=True)

### Build the Network

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x)
        x = self.fc2(x)
        return x

### Init hyperparameters

In [5]:
params = {
    'epochs': 10,
    'learning_rate': 0.001,
    'momentum': 0.9
}

torch.manual_seed(489);

### Init SummaryWriter

In [6]:
log_dir = '/home/hbzahid/runs/exp:'
for param, param_val in sorted(params.items()):
    log_dir += ' {}: {}'.format(param, param_val)
print(log_dir)

# Delete previous log files with same parameters, if any
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

tb = SummaryWriter(log_dir = log_dir)

/home/hbzahid/runs/exp: epochs: 10 learning_rate: 0.001 momentum: 0.9


### Training

In [7]:
net = Net().to(device)
optimizer = optim.SGD(net.parameters(), lr = params['learning_rate'], momentum = params['momentum'])

In [8]:
def dataset_accuracy(loader):
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [9]:
running_loss = 0.

for epoch in range(params['epochs']):
    for batch_num, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    accuracy = dataset_accuracy(train_loader)
    tb.add_scalar('Loss', running_loss, epoch)
    tb.add_scalar('Accuracy', accuracy, epoch)
    tb.add_histogram('conv1.weight', net.conv1.weight, epoch)
    running_loss = 0.

### Test

In [10]:
test_accuracy = dataset_accuracy(test_loader)
print("Test accuracy: ", test_accuracy)

Test accuracy:  0.9401


### Visualize graph

In [10]:
images, labels = next(iter(train_loader))
images, labels = images.to(device), labels.to(device)
grid = torchvision.utils.make_grid(images)
tb.add_image('images', grid)
tb.add_graph(net, images.to(device))

In [11]:
tb.close()